# AlaTB example for ff03-star force field 

In [1]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

### Read GMX files
Read topology and coordinates from GMX files

In [2]:
gro = GromacsGroFile('alaTB_ff03-star_tip3p_solv.gro')

In [3]:
top = GromacsTopFile('alaTB_ff03-star_tip3p.top', \
        periodicBoxVectors=gro.getPeriodicBoxVectors(),
        includeDir='.')

### Simulation workflow
Here we will start with a standard simulation workflow, by first energy minimizing, then running short NVT with constraints, followed by a short NPT run and a production run.

In [4]:
system = top.createSystem(nonbondedMethod=PME, \
        nonbondedCutoff=1*nanometer,
        constraints=HBonds)

#### Defining position restraints
We add position restraints in the heavy atoms using the ```CustomExternalForce``` function.

In [5]:
force = CustomExternalForce(\
            "k*((x - x0)^2 + (y - y0)^2 + (z - z0)^2)")
#force.addGlobalParameter("k", 1000.0*kilojoules_per_mole/nanometer**2)
force.addPerParticleParameter("k")
force.addPerParticleParameter("x0")
force.addPerParticleParameter("y0")
force.addPerParticleParameter("z0")
for i, atom_crd in enumerate(gro.positions):
    k = 1000.0*kilojoules_per_mole/nanometer**2
    x0 = atom_crd[0]
    y0 = atom_crd[1]
    z0 = atom_crd[2]
    if (gro.atomNames[i][0] in ('C', 'N', 'O')) \
            and (gro.atomNames[i][0] not in ('OW')):
        force.addParticle(i,[k, x0, y0, z0])
    else:
        force.addParticle(i,[0., x0, y0, z0])
system.addForce(force)

5

Next we generate the integrator with a ```LangevinIntegrator``` object and pass it on to a ```Simulation``` object. 

In [6]:
integrator = LangevinIntegrator(300*kelvin, \
                        1/picosecond, \
                        0.002*picoseconds)

In [7]:
nvt_posre = Simulation(top.topology, system, \
                        integrator)

We use the gro file atomic positions as initial positions for the simulation.

In [8]:
nvt_posre.context.setPositions(gro.positions)

First, we run a quick energy minimization.

In [9]:
nvt_posre.minimizeEnergy()

We define the reporters for both time series of conformations and also for energies.

In [10]:
pdb_rep = PDBReporter('nvt_posre.pdb', 1000)
nvt_posre.reporters.append(pdb_rep)
state_rep = StateDataReporter(stdout, 100, \
                step=True, potentialEnergy=True, \
                temperature=True, volume=True, density=True)
nvt_posre.reporters.append(state_rep)

In [11]:
nvt_posre.step(1000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)","Density (g/mL)"
100,-32543.243080615124,52.44992245496462,20.679574978308164,0.9753715458022438
200,-31801.852455615124,97.84968905979615,20.679574978308164,0.9753715458022438
300,-31191.157143115124,132.1234598921222,20.679574978308164,0.9753715458022438
400,-30649.461830615124,161.63770216356016,20.679574978308164,0.9753715458022438
500,-30220.586830615124,195.73313648944534,20.679574978308164,0.9753715458022438
600,-29750.946205615124,211.51257211366456,20.679574978308164,0.9753715458022438
700,-29295.071205615124,220.41341649378478,20.679574978308164,0.9753715458022438
800,-29032.883705615124,238.60597043262285,20.679574978308164,0.9753715458022438
900,-28695.204018115124,247.2974384165462,20.679574978308164,0.9753715458022438
1000,-28468.758705615124,256.6906649532725,20.679574978308164,0.9753715458022438


#### NPT run
First we remove the position restrains from the system, because we have already equilibrated the water.

In [12]:
nforces = len(system.getForces())
system.removeForce(nforces-1)

In [13]:
state = nvt_posre.context.getState(getPositions=True, \
                    getVelocities=True)


In [14]:
barostat = MonteCarloBarostat(1.0*bar, 300.0*kelvin, 25)

In [15]:
system.addForce(barostat)

5

In [16]:
integrator = LangevinIntegrator(300*kelvin, \
                        10/picosecond, \
                        0.002*picoseconds)

In [17]:
npt = Simulation(top.topology, system, \
                        integrator)
pdb_rep = PDBReporter('npt.pdb', 1000)
nvt_posre.reporters.append(pdb_rep)

npt.context.setState(state)
npt.reporters.append(state_rep)
npt.step(1000)

100,-27480.11390637525,288.8583277164115,20.569412992987576,0.9805952663793656
200,-27219.089247830037,291.45984945987595,20.81380538584496,0.9690812727038957
300,-27053.487926936505,293.10864512971654,21.058021981813532,0.9578425281607934
400,-26985.222066347662,300.8917173011164,20.69588002741621,0.9746031087542985
500,-26927.01545281656,300.16015974727213,20.631099146019988,0.977663326145032
600,-26772.378134201543,297.2890941826952,20.537992991754926,0.9820954277870911
700,-26857.422174217383,297.78211605837504,20.56740236399136,0.9806911274531775
800,-26589.385887478245,291.2499369362035,20.56967004885076,0.9805830120378018
900,-27188.192164663167,305.48801024000727,20.496566740337286,0.9840803715400152
1000,-26777.600909477012,306.64191878183414,20.45377541857738,0.986139165036786


### Production NVT

In [18]:
nforces = len(system.getForces())
system.removeForce(nforces-1)

In [19]:
state = npt.context.getState(getPositions=True, \
                    getVelocities=True)

In [20]:
integrator = LangevinIntegrator(300*kelvin, \
                        10/picosecond, \
                        0.002*picoseconds)

In [21]:
nvt = Simulation(top.topology, system, \
                        integrator)
dcd_rep = DCDReporter('nvt.dcd', 1000)
nvt.reporters.append(dcd_rep)
#pdb_rep = PDBReporter('nvt.pdb', 1000)
#nvt.reporters.append(pdb_rep)
state_rep = StateDataReporter("nvt.txt", 100, \
                step=True, potentialEnergy=True, \
                temperature=True, volume=True, density=True)
nvt.reporters.append(state_rep)

nvt.context.setState(state)
nvt.step(100000)